In [5]:
import six.moves.urllib.request as urlreq
from six import PY3
import json
import dash
import dash_bio as dashbio
import dash_html_components as html
import json
import six.moves.urllib.request as urlreq
from six import PY3
from dash_bio_utils import circos_parser as cp

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'circos_graph_data.json'
).read()

model_data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'mol3d/model_data.js'
).read()


styles_data = urlreq.urlopen(
    'https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/' +
    'mol3d/styles_data.js'
).read()


if PY3:
    model_data = model_data.decode('utf-8')
    styles_data = styles_data.decode('utf-8')
    data = data.decode('utf-8')

model_data = json.loads(model_data)
styles_data = json.loads(styles_data)
circos_graph_data = json.loads(data)



In [6]:
from Bio import SeqIO

In [7]:
#make circos features

#chrom	chromStart	chromEnd	name	gieStain
#chr1	12700000	16200000	p36.21	gpos50

vals = 'gneg gpos50 gpos75 gpos25 gpos100 acen gvar stalk'.split()

annotationfile = './coronaGenoWuhan.gb'
parser = SeqIO.parse( annotationfile , 'gb')


#color the orfs 

#

#add the blast results to the circos 
with open('corona_circos.txt' , 'w') as circout:
    circout.write('#chrom	chromStart	chromEnd	name	gieStain	color')
    for entry in parser:
        for feature in entry.features:
            if 'product' in feature.qualifiers:
                line = 'chr1\t' + str(feature.location.start)+'\t' + str(feature.location.end) +'\t' + feature.qualifiers['product'][0] + '\t' + vals[1] + '\t #CC0000'
                circout.write(line )
                
json_circos = cp.txt_to_track('corona_circos.txt')
print(json_circos)




[]


In [9]:

import pandas as pd
from itertools import combinations
import numpy as np
from colour import Color
count = 0

annotationfile = './coronaGenoWuhan.gb'
parser = SeqIO.parse( annotationfile , 'gb')


### stack graph
for entry in parser:
    for feature in entry.features:
        if 'product' in feature.qualifiers:
            count +=1

#{'chr': 'chr9', 'start': 2728594, 'end': 2948764, 'block_id': 'chr9'}
color1 = Color('red')
color2 = Color('yellow')
print(count)
colorvec = color1.range_to(color2, count+1)
colorvec = [ c.hex_l for c in colorvec ]
print(colorvec)
###layout fmt
#{'id': 'chr1', 'label': 'chr1', 'color': '#996600', 'len': 249250621 }
circos_graph_data = {}
circos_graph_data['layout'] = [{'id': 'chr1', 'label':'' , 'len': 29674 , 'color':'#CC0000' } ]
circos_graph_data['stack'] = []
parser = SeqIO.parse( annotationfile , 'gb')

count = 0
featlist = []
for entry in parser:
    for feature in entry.features:
        if 'product' in feature.qualifiers:
            count +=1
            if feature.qualifiers['product'][0] not in featlist:
                featlist.append(feature.qualifiers['product'][0])
                circos_graph_data['stack'].append( {'chr': feature.qualifiers['product'][0], 'block_id':'chr1' , 'start': feature.location.start , 'end':  feature.location.end , 'color':colorvec[count] } )    
clusterdf = pd.read_csv( '../lucy_mk2/30_07_2020/gisaid_hcov-2020_07_30.QC.NSoutlier.filter.deMaiomask.EPIID.HF.noambig.aln_svd_codon_clustersclusterpositions_columns.csv')

#one color per cluster
for i,r in clusterdf.iterrows():    
    rowpos = [ int(entry) for entry in r if not np.isnan(entry) ]
    circos_graph_data[i] = [ {'color': '#ff5722', 'source': {'id': 'chr1', 'start': i, 'end': i+2 }, 'target': {'id': 'chr1', 'start': j, 'end': j+2}} for i,j in combinations(rowpos,2) ] 


38
['#ff0000', '#ff0700', '#ff0d00', '#ff1400', '#ff1b00', '#ff2200', '#ff2800', '#ff2f00', '#ff3600', '#ff3c00', '#ff4300', '#ff4a00', '#ff5100', '#ff5700', '#ff5e00', '#ff6500', '#ff6b00', '#ff7200', '#ff7900', '#ff7f00', '#ff8600', '#ff8d00', '#ff9400', '#ff9a00', '#ffa100', '#ffa800', '#ffae00', '#ffb500', '#ffbc00', '#ffc300', '#ffc900', '#ffd000', '#ffd700', '#ffdd00', '#ffe400', '#ffeb00', '#fff200', '#fff800', '#ffff00']


In [10]:
models = '5x58,6cs0,6sc1,6nb6, 2jw8, 2xab, 4aud1, 1yo4, 2acf, 2wct, 3vc8, 2gt7, 3ee7, 2g9t, 3ee7, 2g9t, 6jyt, 1ysy, 6nur, 2g9t, 5c8u, 2g9t, 2xyq, 4mm3, 6cs2, 6acg, 6acj, 6ack, 2dd8, 2ghw, 6nb6, 6nb7'
models = models.split(',')
templatedir = './templates/'

#map structs to geno. 

#add track on geno

#dict from geno 2 struct residue

#

In [11]:
import os
import wget
import requests
import glob
import time
clear = False

In [12]:
if not os.path.isdir(templatedir):
    os.mkdir(templatedir)
if clear == True:
    files = glob.glob(templatedir+'*.pdb')
    for f in files:
        os.remove(f)

In [24]:
from Bio.PDB import *
dl_url = 'http://files.rcsb.org/download/'
dl_url_err = 'http://files.rcsb.org/download/'

structs = {}
already = glob.glob( './templates/*.pdb' )
print(already)
#pull complexes
for m in models:
    structfile = './templates/'+m.upper().strip()+'.pdb'
    if structfile not in already:
        print(m)
        time.sleep(1)
        try:
            wget.download(url = dl_url + m.strip() +'.pdb' , out =structfile)
            structs[m] = structfile
        except:
            try:
                wget.download(url = dl_url + m.strip() +'.pdb' , out =structfile)
                structs[m] = structfile
            except:
                print('err', m )
    else:
        structs[m.strip()] = structfile


['./templates/6ACG.pdb', './templates/6NUR.pdb', './templates/5C8U.pdb', './templates/2DD8.pdb', './templates/2WCT.pdb', './templates/6ACJ.pdb', './templates/2G9T (1).pdb', './templates/6NB6.pdb', './templates/5X58.pdb', './templates/6NB7.pdb', './templates/6JYT.pdb', './templates/2GT7.pdb', './templates/3VC8.pdb', './templates/3EE7.pdb', './templates/4MM3.pdb', './templates/2ACF.pdb', './templates/2G9T.pdb', './templates/2XYQ.pdb', './templates/6NB6 (1).pdb', './templates/6CS2.pdb', './templates/6CS0.pdb', './templates/1YSY.pdb', './templates/1YO4.pdb', './templates/2XAB.pdb', './templates/6ACK.pdb', './templates/2GHW.pdb', './templates/2JW8.pdb', './templates/3EE7 (1).pdb']
6sc1
err 6sc1
 4aud1
err  4aud1


In [25]:
print(structs)

{'5x58': './templates/5X58.pdb', '6cs0': './templates/6CS0.pdb', '6nb6': './templates/6NB6.pdb', '2jw8': './templates/2JW8.pdb', '2xab': './templates/2XAB.pdb', '1yo4': './templates/1YO4.pdb', '2acf': './templates/2ACF.pdb', '2wct': './templates/2WCT.pdb', '3vc8': './templates/3VC8.pdb', '2gt7': './templates/2GT7.pdb', '3ee7': './templates/3EE7.pdb', '2g9t': './templates/2G9T.pdb', '6jyt': './templates/6JYT.pdb', '1ysy': './templates/1YSY.pdb', '6nur': './templates/6NUR.pdb', '5c8u': './templates/5C8U.pdb', '2xyq': './templates/2XYQ.pdb', '4mm3': './templates/4MM3.pdb', '6cs2': './templates/6CS2.pdb', '6acg': './templates/6ACG.pdb', '6acj': './templates/6ACJ.pdb', '6ack': './templates/6ACK.pdb', '2dd8': './templates/2DD8.pdb', '2ghw': './templates/2GHW.pdb', '6nb7': './templates/6NB7.pdb'}


In [26]:
#open all the complexes and pull sequences
#convert to json
from Bio import Alphabet
parser = PDBParser()
from Bio.SeqUtils import seq1
letter3 = Alphabet.ThreeLetterProtein.letters
converter = { l.upper(): seq1(l) for l in letter3 }
print(converter)

{'ALA': 'A', 'ASX': 'B', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L', 'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R', 'SER': 'S', 'THR': 'T', 'SEC': 'U', 'VAL': 'V', 'TRP': 'W', 'XAA': 'X', 'TYR': 'Y', 'GLX': 'Z'}


In [27]:
import subprocess
import shlex
import pandas as pd 
#collapse codons
#use tblastn to map to prot

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder
ppb=PPBuilder()

with open( 'structs.fast' , 'w')as fastout:
    for s in structs:
        Structure = PDBParser().get_structure(s, structs[s])
        for model in Structure:
            for chain in model:
                for i, pp in enumerate(ppb.build_peptides(chain)):
                    fastout.write('>' + s + '|'+ chain.id +'|'+str(i)+'\n')
                    fastout.write(str( pp.get_sequence() ) +'\n'  )


def makeblastDB(fasta , blastdb):
    print(fasta)
    #make a blast database with the structures
    args = 'makeblastdb -in '+ fasta +' -out '+ blastdb + ' -dbtype prot'
    p = subprocess.run( shlex.split(args) )
    return p ,  blastdb


def runblastx( qfile ,db ,  outannot = 'outannot.txt' , outfmt = None ):
    if outfmt is None:
        outfmt = [ 'qseqid' , 'sseqid' , 'qlen' ,  'slen' , 'qstart' , 'qend' ,  'qframe' , 'evalue' ]
        outfmt =  ' "10 ' + ''.join([fmt+ ' ' for fmt in outfmt]) + ' " '
        print(outfmt)
        print(db)
    args = 'blastx -query '+ qfile + ' -db '+ db +' -outfmt' + outfmt + ' -out ' + outannot  
    p = subprocess.run( shlex.split(args) )
    return p , outannot


out = 'outannot.txt'
p,blastdb = makeblastDB( './structs.fast' , 'structDB' )
p,annot = runblastx( 'covid19geno.fasta' , blastdb )

annotation = pd.read_csv( annot , header = None )
annotation.columns = [ 'qseqid' , 'sseqid' , 'qlen' ,  'slen' , 'qstart' , 'qend' ,  'qframe' , 'evalue' ] 
annotation = annotation[ annotation['evalue'] < 10**-3 ]
annotation = annotation[annotation.evalue < 10 ** -20 ]
annotation = annotation.drop_duplicates(['sseqid'])
annotation['struct'] = annotation.sseqid.map(lambda x : x.split('|')[0])
annotation['chain'] = annotation.sseqid.map(lambda x : x.split('|')[1])
print(annotation)

print(len(annotation), ' orfs detected')
#recluster codons
#load blast results into df




/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning:


/home/cactuskid13/miniconda3/envs/pyprofiler3/lib/python3.7/site-packages/Bi

./structs.fast
 "10 qseqid sseqid qlen slen qstart qend qframe evalue  " 
structDB
          qseqid    sseqid   qlen  slen  qstart   qend  qframe        evalue  \
0    NC_045512.2  6nur|A|0  29903   779   13789  16125       1  0.000000e+00   
1    NC_045512.2  5c8u|D|0  29903   453   18040  19398       1  0.000000e+00   
2    NC_045512.2  5c8u|B|0  29903   453   18040  19398       1  0.000000e+00   
3    NC_045512.2  5x58|C|1  29903   419   22334  23590       2  0.000000e+00   
4    NC_045512.2  5x58|B|1  29903   419   22334  23590       2  0.000000e+00   
..           ...       ...    ...   ...     ...    ...     ...           ...   
280  NC_045512.2  6jyt|B|2  29903    40   16936  17055       1  4.740000e-22   
281  NC_045512.2  6jyt|A|2  29903    40   16936  17055       1  4.740000e-22   
282  NC_045512.2  2g9t|V|1  29903    37   13169  13279       2  2.250000e-21   
283  NC_045512.2  2g9t|F|1  29903    37   13169  13279       2  2.250000e-21   
284  NC_045512.2  2g9t|U|1  29903    

In [28]:
count = 0
circos_graph_data['struct_stack'] = []
for i,r in annotation.iterrows():
    circos_graph_data['struct_stack'].append( {'chr': r.sseqid , 'block_id':'chr1' , 'start': r.qstart , 'end':  r.qend  } ) 
    count +=1

In [29]:
import os
import base64
import json
import tempfile

from shutil import copy2
from textwrap import dedent as s

from dash.dependencies import Input, Output, State
import dash_html_components as html
import dash_core_components as dcc
import dash_daq as daq
from dash_bio_utils import pdb_parser as parser, styles_parser as sparser
import dash_bio


def files_data_style(content):
    fdat = tempfile.NamedTemporaryFile(suffix=".js", delete=False, mode='w+')
    fdat.write(content)
    dataFile = fdat.name
    fdat.close()
    return dataFile

# Create the model data from the decoded contents
fname = structs[list(structs.keys())[3]]
modata = parser.create_data(fname)
fmodel = files_data_style(modata)
with open(fmodel) as fm:
    mdata = json.load(fm)

datstyle = sparser.create_style(fname, 'cartoon', 'residue')

fstyle = files_data_style(datstyle)

with open(fstyle) as sf:
    data_style = json.load(sf)

In [30]:
for atom_id in mdata['atoms'][0:25]:
    print(atom_id)
for atom_id in mdata['atoms'][-25:]:
    print(atom_id)
print( data_style['1'] )
maxnum = 0
for serial in data_style:
    if int(serial)>maxnum :
        maxnum = int(serial)
maxserial = 0
for atom_id in mdata['atoms']:
    if int(atom_id['serial'])>maxserial :
        maxserial = int(atom_id['serial'])
        
print(maxnum)
print(maxserial)

{'name': 'N', 'chain': 'A', 'positions': [30.514, 14.219, 2.014], 'residue_index': 1, 'element': 'N', 'residue_name': 'THR248', 'serial': 0}
{'name': 'CA', 'chain': 'A', 'positions': [29.356, 13.316, 1.944], 'residue_index': 1, 'element': 'C', 'residue_name': 'THR248', 'serial': 1}
{'name': 'C', 'chain': 'A', 'positions': [28.082, 14.164, 1.848], 'residue_index': 1, 'element': 'C', 'residue_name': 'THR248', 'serial': 2}
{'name': 'O', 'chain': 'A', 'positions': [28.175, 15.387, 1.95], 'residue_index': 1, 'element': 'O', 'residue_name': 'THR248', 'serial': 3}
{'name': 'CB', 'chain': 'A', 'positions': [29.354, 12.378, 3.163], 'residue_index': 1, 'element': 'C', 'residue_name': 'THR248', 'serial': 4}
{'name': 'OG1', 'chain': 'A', 'positions': [29.706, 13.115, 4.319], 'residue_index': 1, 'element': 'O', 'residue_name': 'THR248', 'serial': 5}
{'name': 'CG2', 'chain': 'A', 'positions': [30.366, 11.237, 2.997], 'residue_index': 1, 'element': 'C', 'residue_name': 'THR248', 'serial': 6}
{'name':

In [31]:
def set_color_cluster(structfile, cluster , mdata ):
    #blue and yellow high contrast
    #find match in annotation and blast ouput
    #geno positions from cluster file
    positions = [ int(p) for p in clusterdf.loc[cluster] if ~np.isnan(p) ]
    #select struct
    geno_annot = annotation[ structfile == annotation.struct ]
    residues = {}
    for i,r in geno_annot.iterrows():
        if r.chain not in residues:
            residues[r.chain] = []
        for p in positions:
            if p < r.qend and p > r.qstart:
                print(p)
                #match codon to amino acid position
                #todo: triple check this!!!!
                residues[r.chain].append( int( p - r.qstart / 3 ) )
    
    #map from geno pos to residue index
    colors =[]
    #intersect both
    #list comp to generate residue positions
    for atom_data in mdata['atoms']:
        for chain in residues:
            if atom_data['chain'] == chain  and atom_data['residue_index'] in residues[chain]:
                #color select
                #set the sidechains to stick view
                colors.append({'color': '#CC0000', 'visualization_type': 'stick'})
            else:
                #color bg 
                colors.append({'color': '#00CCCC', 'visualization_type': 'cartoon'})
    return colors

colors = set_color_cluster( list(structs.keys())[3], 20 , mdata)
print(colors[0:10])

[{'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}, {'color': '#00CCCC', 'visualization_type': 'cartoon'}]


In [32]:
print(list(data_style.keys())[0:10])
print(data_style['0'])

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
{'color': '#FA9600', 'visualization_type': 'cartoon'}


In [33]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

data_info = { os.path.abspath( structs[s])  : {
        'name': 'covid prot'+str(i),
        'description': 'test description',
        'link':'test link' } for i,s in enumerate(structs) }
stacktrack = {
                    'type': 'STACK',
                    'data': circos_graph_data['stack'],
                    'config': {
                        'innerRadius': .9 ,
                        'outerRadius': 1.2,
                        'thickness': 4,
                        'margin': 0,
                        'direction': 'out',
                        'strokeWidth': 0,
                        'opacity': 0.5,
                        'tooltipContent': {'name': 'chr'}
                        ,'color': {
                            'conditional': {
                                'end': 'end',
                                'start': 'start',
                                'value': [ n for n in range( 30000 , 1000)  ],
                                'color': [
                                    'red',
                                    'black',
                                    '#fff',
                                    '#999',
                                    '#BBB',
                                ],
                            }
                            
                        }
                }
}


structtrack = {
                    'type': 'STACK',
                    'data': circos_graph_data['struct_stack'],
                    'config': {
                        'innerRadius': 0.7,
                        'outerRadius': 0.9,
                        'thickness': 4,
                        'margin': 0,
                        'direction': 'out',
                        'strokeWidth': 0,
                        'opacity': 0.5,
                        'tooltipContent': {'name': 'chr'}
                        }
                }
def header_colors():
    return {
        'bg_color': '#e7625f',
        'font_color': 'white'
    }

def description():
    return 'Molecule visualization in 3D - perfect for viewing ' \
           'biomolecules such as proteins, DNA and RNA. Includes ' \
           'stick, cartoon, and sphere representations.'

app.layout = html.Div(
        id='mol3d-body',
        className = 'row',
        children=[
            
######################### circos control pannel #####################################
        html.Div([ 
       
        html.Div([ 
        dashbio.Circos(
        id='my-dashbio-circos',
        layout= circos_graph_data['layout'],
        selectEvent={"0": "hover", "1": "click", "2": "both"},
        tracks=[
           
            {
            'type': 'CHORDS',
            'data': circos_graph_data[0],
                
            'config': {
                'tooltipContent': {
                    'source': 'source',
                    'sourceID': 'id',
                    'target': 'target',
                    'targetID': 'id',
                    'targetEnd': 'end'
                }
            }
        }
            ,
            stacktrack     
            ,
            structtrack
            
    
        ] ) 

        ], id = 'circos' , className="six columns"  )  
            
       
        
        ,

        dcc.Store(
                id='mol3d-color-storage',
                data={}
            ),
    
        html.Div(
        id='mol3d-biomolecule-viewer',
        children= [
            
############################# 3d viewer ####################################################
        dash_bio.Molecule3dViewer(
        id='mol-3d',
        selectionType='atom',
        modelData=mdata,
        styles = data_style ,
        selectedAtomIds=[],
        backgroundOpacity='0',
        atomLabelsShown=False,
        )

        
        ] , className="six columns")
        ,

####################### control pannel ############################################
                    
        html.Div( className="six columns" , children = [
        
            html.Div(
                id='mol3d-control-tabs',
                className='control-tabs',
                children=[
                    dcc.Tabs(id='mol3d-tabs', value='what-is', children=[
                        dcc.Tab(
                            label='cluster select',
                            value='what-is',
                            children=html.Div(className='control-tab', children=[    
                            "Graph type:",
                            dcc.Dropdown(
                                id='histogram-chords',
                                options=[
                                    {'label': x, 'value': x}
                                    for x in list(circos_graph_data.keys()) if type(x) is int
                                    ],
                                value=0
                                        ),
                                
                            "Event data:",    
                            html.Div(id='circos-output')
                                
                            ] )
                        )
                        
                        ,
                        
                        dcc.Tab(
                            label='Structure select',
                            value='upload-download',
                            children=html.Div(className=' six columns', children=[
                                html.Div(
                                    title='Select molecule to view',
                                    className="app-controls-block",
                                    children=[
                                        html.Div(className='app-controls-name',
                                                 children='Select structure'),
                                        dcc.Dropdown(
                                            id='dropdown-demostr',
                                            options=[ { 'label':s,  'value':os.path.abspath(structs[s]) } for s in structs ],
                                            value= os.path.abspath(structs[list(structs.keys())[3]])
                                        ),
                                    ],
                                ),
                            html.Div(id='mol3d-data-info')
                            ])
                        ),

                        dcc.Tab(
                            label='Structure Info and coloring',
                            value='view-options',
                            children=html.Div(className='control-tab', children=[
                                # Textarea container to display the selected atoms
                                html.Div(
                                    title='view information about selected atoms '
                                    'of biomolecule',
                                    className="app-controls-block",
                                    id="mol3d-selection-display",
                                    children=[
                                        html.P(
                                            "Selection",
                                            style={
                                                'font-weight': 'bold',
                                                'margin-bottom': '10px'
                                            }
                                        ),
                                        html.Div(id='mol3d-selection-output'),
                                    ]
                                ),
                                # Dropdown to select chain representation
                                # (sticks, cartoon, sphere)
                                html.Div(
                                    title='select style for molecule representation',
                                    className="app-controls-block",
                                    id='mol3d-style',
                                    children=[
                                        html.P(
                                            'Style',
                                            style={
                                                'font-weight': 'bold',
                                                'margin-bottom': '10px'
                                            }
                                        ),
                                        dcc.Dropdown(
                                            id='dropdown-styles',
                                            options=[
                                                {'label': 'Sticks', 'value': 'stick'},
                                                {'label': 'Cartoon', 'value': 'cartoon'},
                                                {'label': 'Spheres', 'value': 'sphere'},
                                            ],
                                            value='cartoon'
                                        ),
                                    ],
                                ),

                                # Dropdown to select color of representation
                                html.Div(
                                    title='select color scheme for viewing biomolecule',
                                    className="app-controls-block",
                                    id='mol3d-style-color',
                                    children=[
                                        html.P(
                                            'Color',
                                            style={
                                                'font-weight': 'bold',
                                                'margin-bottom': '10px'
                                            }
                                        ),
                                        dcc.Dropdown(
                                            id='dropdown-style-color',
                                            options=[
                                                {'label': 'Atom',
                                                 'value': 'atom'},
                                                {'label': 'Residue identity',
                                                 'value': 'residue'},
                                                {'label': 'Residue type',
                                                 'value': 'residue_type'},
                                                {'label': 'Chain',
                                                 'value': 'chain'},
                                            ],
                                            value='residue'
                                        ),
                                        dcc.Dropdown(
                                            id='mol3d-coloring-key',
                                            options=[]
                                        ),

                                    ],
                                ),
                                html.Div(
                                    title='Customize molecule coloring.',
                                    className="app-controls-block",
                                    children=[
                                        html.P(
                                            id='mol3d-customize-coloring',
                                            style={
                                                'font-weight': 'bold',
                                                'margin-bottom': '10px'
                                            }
                                        ),
                                        daq.ColorPicker(
                                            id='mol3d-coloring-value',
                                            size=315
                                        ),
                                    ]
                                ),
                            ]),
                        ),

                    ]),
                ]  ),
        ])
        ],className="row") , dcc.Store(id='structswitch' )   ])
        


########################circos functions ########################
@app.callback(
    
    Output('circos-output', 'children'),
    [Input('my-dashbio-circos', 'eventDatum')]
    
)
def update_output(value):
    if value is not None:
        return [html.Div('{}: {}'.format(v.title(), value[v]))
                for v in value.keys()]
    return 'There are no event data. Click or hover on a data point to get more information.'

####change the circos content
@app.callback(
    Output('my-dashbio-circos', 'tracks'),
    [Input('histogram-chords', 'value')],
    state=[State('my-dashbio-circos', 'tracks')]
)
def change_graph_type(value, current):   
    current[0].update(
            {
                
            'type': 'CHORDS',
            'data': circos_graph_data[int(value)] ,
            'config': {
                'tooltipContent': {
                    'source': 'source',
                    'sourceID': 'id',
                    'target': 'target',
                    'targetID': 'id',
                    'targetEnd': 'end'
                }
            }
        }   
    )
    current[1].update(stacktrack)
    current[2].update(structtrack)
    return current
            

######### Function to create the modelData and style files for molecule visualization
@app.callback(
    Output('mol3d-data-info', 'children'),
    [Input('dropdown-demostr', 'value')]
)
def show_data(molecule_selected):
    if molecule_selected in data_info.keys():
        mol = data_info[molecule_selected]
        return [
            html.H4(mol['name']),
            mol['description'],
            html.A(
                '(source)',
                href=mol['link']
            )
        ]
    return ''




# Callback for updating dropdown options
@app.callback(
    Output('mol3d-coloring-key', 'options'),
    [Input('dropdown-style-color', 'value')]
)
def update_color_options(mol_style):
    color_dict_keys = {
        'atom': list(sparser.ATOM_COLOR_DICT.keys()),
        'residue': list(sparser.RESIDUE_COLOR_DICT.keys()),
        'residue_type': list(sparser.RESIDUE_TYPE_COLOR_DICT.keys()),
        'chain': list(sparser.CHAIN_COLOR_DICT.keys())
    }
    options = [{'label': k.upper(), 'value': k}
               for k in color_dict_keys[mol_style]]
    return options

@app.callback(
    [Output('structswitch','children'),Output("structswitch", "data")],  
    [ Input('dropdown-demostr', 'value'),Input('histogram-chords', 'value') , Input('mol-3d' , 'modelData') ] ,
)
def set_color_cluster( structfile, cluster , mdata ):
    #blue and yellow high contrast
    #find match in annotation and blast ouput
    #geno positions from cluster file
    positions = [ int(p) for p in clusterdf.loc[cluster] if ~np.isnan(p) ]
    #select struct
    print(structfile)
    struct = structfile.split('/')[-1].replace('.pdb','').lower()
    geno_annot = annotation[ struct == annotation.struct ]
    print(geno_annot)
    residues = {}
    for i,r in geno_annot.iterrows():
        if r.chain not in residues:
            residues[r.chain] = []
        for p in positions:
            if p < r.qend and p > r.qstart:
                print(p)
                #match codon to amino acid position
                #todo: triple check this!!!!
                residues[r.chain].append( int( (p - r.qstart) / 3 ) )
    
    print(residues)
    
    currentselection = []
    chaindiffs = {}
    chaincount = 0
    
    print('chain offset' , chaindiffs)
    
    for atom_data in mdata['atoms']:
        for chain in residues:
            ###todo : triple check this
            if atom_data['chain'] == chain  and int(atom_data['residue_name'][3:]) in residues[chain]:
                currentselection.append(atom_data['serial'])
            #color select
    print('selection', currentselection)
    return structfile , currentselection

# Callback for molecule visualization based on uploaded PDB file
@app.callback(
    Output('mol3d-biomolecule-viewer', 'children'),
    [Input("structswitch", "data") , Input('dropdown-demostr', 'value'),
    Input('dropdown-styles', 'value'),
    Input('dropdown-style-color', 'value'),
    Input('mol3d-color-storage', 'modified_timestamp')],
    [State('mol3d-color-storage', 'data') ],
)
def use_upload( selection , demostr, mol_style, color_style, mt, custom_colors  ):
    print(selection)
    fname = demostr
    
    copy2(demostr, './str.pdb')
    fname = './str.pdb'
    # Create the model data from the decoded contents
    
    print(custom_colors)
    modata = parser.create_data(fname)
    fmodel = files_data_style(modata)
    with open(fmodel) as fm:
        mdata = json.load(fm)
    #all blue
    datstyle = sparser.create_style(fname, mol_style, color_style, **custom_colors)
    fstyle = files_data_style(datstyle)
    with open(fstyle) as sf:
        data_style = json.load(sf)
    
    print('selection finish' , selection)
    data_style = [ {'color': 'blue', 'visualization_type': 'cartoon'}  if i not in selection else  {'color': 'yellow', 'visualization_type': 'cartoon'}  for i,atom in enumerate(data_style) ]
    
    # Return the new molecule visualization container
    return dash_bio.Molecule3dViewer(
        id='mol-3d',
        selectionType='atom',
        modelData=mdata,
        styles=data_style,
        selectedAtomIds= [] ,
        backgroundOpacity='0',
        atomLabelsShown=False,
        )

#### add struct select callback from circos#######
# Callback to print details of each selected atom of the biomolecule
@app.callback(
    Output("mol3d-selection-output", "children"),
    [Input("mol-3d", "selectedAtomIds"),
     Input("mol-3d", "modelData")]
)
def selout(selected_atom_ids, model_data):
    residue_summary = []
    for atom_id in selected_atom_ids:
        res_info = model_data['atoms'][atom_id]
        residues = {
            "residue": res_info['residue_name'],
            "atom": res_info['name'],
            "chain": res_info['chain'],
            "xyz": res_info['positions']}
        
        residue_summary += [html.P('{}: {}'.format(
            key, str(residues[key]))) for key in residues]
        residue_summary.append(html.Br())
    if len(residue_summary) == 0:
        residue_summary.append("No atoms have been selected. Click \
        on an atom to select it.")
    return html.Div( residue_summary )

In [ ]:
app.run_server( port=8050 )

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
